# Import required libraries


In [ ]:
import random
import re
from pprint import pprint

import pandas as pd
import requests
import stanza

# Import all sheets from the 7000 Sentences dataset


In [ ]:
df_7000_sents_1 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="3000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

df_7000_sents_2 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="6000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

df_7000_sents_3 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="1000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

In [ ]:
df_7000_sents_1 = df_7000_sents_1.dropna().reset_index(drop = True)
df_7000_sents_2 = df_7000_sents_2.dropna().reset_index(drop = True)
df_7000_sents_3 = df_7000_sents_3.dropna().reset_index(drop = True)

# Import the 3000 Words dataset


In [ ]:
df_3000_words = pd.read_excel(
    "../data/corpus/3000 Hindi Words Corpus With IDs.xlsx",
    sheet_name="Feuille1",
    usecols=["ID", "French Word", "Hindi Word"],
    na_values=["NA"],
)

In [ ]:
df_3000_words = df_3000_words.dropna().reset_index(drop = True)

# Download a Stanza Language Model for Hindi and French into the directory "../stanza_models"


In [ ]:
stanza.download(lang='hi')

# Initialize a Stanza pipeline with a language model for Hindi and French

## Which is assigned to the variable 'nlp_hi' and 'nlp_fr' using the Pipeline() class


In [ ]:
nlp_hi_stanza = stanza.Pipeline(lang='hi')

# Set Exercise Dataset Columns

In [ ]:
cols = [
    "Exo_type_id",
    "Exo_type",
    "Exo_objective",
    "Exo_focus",
    "Exo_id",
    "Source_format",
    "Target_format",
    "Source_sentence_id",
    "Source_word_id",
    "Source_lang",
    "Target_lang",
    "Full_sentence",
    "Instruction",
    "Sentence_w_blank",
    "Right_answer",
    "Dist_1",
    "Dist_2",
    "Dist_3",
    "Explanation",
    "Difficulty",
    "Remediation",
]

# Distractor Creation Functions


## Spelling Distractors


In [ ]:
# exchanges two letters in a given word
def exchange_letter(a, b, text, list_of_distractors):
    if (a in text):
        list_of_distractors.append(text.replace(a, b))

    if (b in text):
        list_of_distractors.append(text.replace(b, a))

    return (list_of_distractors)

# Mistakes with similar sounding and looking vowels
def vowel_changer(word, list_of_distractors):
    if ('इ' in word.text or 'ई' in word.text):
        list_of_distractors = exchange_letter(
            'इ', 'ई', word.text, list_of_distractors)

    if ('ि' in word.text or 'ी' in word.text):
        list_of_distractors = exchange_letter(
            'ि', 'ी', word.text, list_of_distractors)

    if ('उ' in word.text or 'ऊ' in word.text):
        list_of_distractors = exchange_letter(
            'उ', 'ऊ', word.text, list_of_distractors)

    if ('ु' in word.text or 'ू' in word.text):
        list_of_distractors = exchange_letter(
            'ु', 'ू', word.text, list_of_distractors)

    if ('ए' in word.text or 'ऐ' in word.text):
        list_of_distractors = exchange_letter(
            'ए', 'ऐ', word.text, list_of_distractors)

    if ('े' in word.text or 'ै' in word.text):
        list_of_distractors = exchange_letter(
            'े', 'ै', word.text, list_of_distractors)

    if ('ओ' in word.text or 'औ' in word.text):
        list_of_distractors = exchange_letter(
            'ओ', 'औ', word.text, list_of_distractors)

    if ('ो' in word.text or 'ौ' in word.text):
        list_of_distractors = exchange_letter(
            'ो', 'ौ', word.text, list_of_distractors)

    if ('अं' in word.text or 'अँ' in word.text):
        list_of_distractors = exchange_letter(
            'अं', 'अँ', word.text, list_of_distractors)

    if ('ं' in word.text or 'ँ' in word.text):
        list_of_distractors = exchange_letter(
            'ं', 'ँ', word.text, list_of_distractors)

    if ('ॉ' in word.text or 'ाँ' in word.text):
        list_of_distractors = exchange_letter(
            'ॉ', 'ाँ', word.text, list_of_distractors)

    if ('ॉ' in word.text or 'ां' in word.text):
        list_of_distractors = exchange_letter(
            'ॉ', ' ां', word.text, list_of_distractors)

    if ('ाँ' in word.text or 'ां' in word.text):
        list_of_distractors = exchange_letter(
            'ाँ', 'ां', word.text, list_of_distractors)

    return list_of_distractors


# Mistakes with similar sounding consonents
def consonent_changer(word, list_of_distractors):
    if ('ट' in word.text or 'त' in word.text):
        list_of_distractors = exchange_letter(
            'ट', 'त', word.text, list_of_distractors)

    if ('ठ' in word.text or 'थ' in word.text):
        list_of_distractors = exchange_letter(
            'ठ', 'थ', word.text, list_of_distractors)

    if ('ड' in word.text or 'द' in word.text):
        list_of_distractors = exchange_letter(
            'ड', 'द', word.text, list_of_distractors)

    if ('ढ' in word.text or 'ध' in word.text):
        list_of_distractors = exchange_letter(
            'ढ', 'ध', word.text, list_of_distractors)

    if ('न' in word.text or 'ण' in word.text):
        list_of_distractors = exchange_letter(
            'न', 'ण', word.text, list_of_distractors)

    if ('श' in word.text or 'ष' in word.text):
        list_of_distractors = exchange_letter(
            'श', 'ष', word.text, list_of_distractors)

    if ('श' in word.text or 'स' in word.text):
        list_of_distractors = exchange_letter(
            'श', 'स', word.text, list_of_distractors)

    if ('स' in word.text or 'ष' in word.text):
        list_of_distractors = exchange_letter(
            'स', 'ष', word.text, list_of_distractors)

    return list_of_distractors

# Mistakes with similar looking letters
def letter_changer(word, list_of_distractors):
    if ('ज' in word.text or 'ज्ञ' in word.text):
        list_of_distractors = exchange_letter(
            'ज', 'ज्ञ', word.text, list_of_distractors)

    if ('क्ष' in word.text or 'श्र' in word.text):
        list_of_distractors = exchange_letter(
            'क्ष', 'श्र', word.text, list_of_distractors)

    return list_of_distractors

# Create 3 spelling distractors for a given word
def spelling_distractors(word, list_of_distractors=None):

    if list_of_distractors is None:
        list_of_distractors = []

    list_of_distractors = vowel_changer(word, list_of_distractors)
    if len(list_of_distractors) < 3:
        list_of_distractors = consonent_changer(word, list_of_distractors)
    
    if len(list_of_distractors) < 3:
        list_of_distractors = letter_changer(word, list_of_distractors)

    if len(list_of_distractors) >= 3:
        list_of_distractors = list_of_distractors[:3]
    else:
        list_of_distractors = []

    return list_of_distractors

# Creating Vocabulary Flashcards


In [ ]:
def word_flashcards(df_source, exo_id = 0):
    data = []
    for i in range(len(df_source)):
        exo_id += 1
        data.append([
            "10",
            "Flashcards",
            "Learning vocabulary",
            "",
            str(exo_id),
            "text",
            "text",
            "",
            str(df_source["ID"][i]),
            "French",
            "Hindi",
            str(df_source["Hindi Word"][i]),
            "",
            "",
            str(df_source["French Word"][i]),
            "",
            "",
            "",
            "",
            "",
            "",
        ])
    return exo_id, data

exo_id = 0

exo_id, data_w_fc_1 = word_flashcards(df_3000_words, exo_id)
df_w_fc_1 = pd.DataFrame(data_w_fc_1, columns=cols)

frames_w_fc = [df_w_fc_1]
df_w_fc = pd.concat(frames_w_fc)
df_w_fc

# Create Spelling MCQ Exercises


In [ ]:
def spelling_mcq(df_source, exo_id, cols):
    data = []

    spellings = {col: [] for col in cols}


    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi Word"][i]))

        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                list_of_distractors = []

                if word.text not in spellings["Right_answer"]:
                    list_of_distractors = spelling_distractors(word, list_of_distractors)

                    if list_of_distractors != []:
                        
                        exo_id += 1

                        spellings["Exo_type_id"].append("14")
                        spellings["Exo_type"].append("MCQ")
                        spellings["Exo_objective"].append("Learning vocabulary")
                        spellings["Exo_focus"].append("Spellings")
                        spellings["Exo_id"].append(str(exo_id))
                        spellings["Source_format"].append("text")
                        spellings["Target_format"].append("text")
                        spellings["Source_sentence_id"].append("")
                        spellings["Source_word_id"].append(str(df_source["ID"][i]))
                        spellings["Source_lang"].append("French")
                        spellings["Target_lang"].append("Hindi")
                        spellings["Full_sentence"].append(str(df_source["Hindi Word"][i]))
                        spellings["Instruction"].append("Select the correct spelling:")
                        spellings["Sentence_w_blank"].append("")
                        spellings["Right_answer"].append(str(word.text))
                        spellings["Dist_1"].append(list_of_distractors[0]),
                        spellings["Dist_2"].append(list_of_distractors[1]),
                        spellings["Dist_3"].append(list_of_distractors[2]),
                        spellings["Explanation"].append("")
                        spellings["Difficulty"].append("")
                        spellings["Remediation"].append("")

    return exo_id, spellings

exo_id = 0

exo_id, data_spelling_mcq_1 = spelling_mcq(df_3000_words, exo_id, cols)
df_spelling_mcq_1 = pd.DataFrame(data_spelling_mcq_1, columns=cols)

frames_spelling_mcq = [df_spelling_mcq_1]
df_spelling_mcq = pd.concat(frames_spelling_mcq)
df_spelling_mcq

# Creating Useful Sentences Flashcards


In [ ]:
def sentence_flashcards(df_source):
    data = []
    exo_id = 0
    for i in range(len(df_source)):
        if df_source["Hindi"][i] != "NaN":
            exo_id += 1
            data.append([
                "24",
                "Flashcards",
                "Useful Sentences",
                "",
                str(exo_id),
                "text",
                "text",
                str(df_source["ID"][i]),
                "",
                "French",
                "Hindi",
                str(df_source["Hindi"][i]),
                "",
                "",
                str(df_source["French"][i]),
                "",
                "",
                "",
                "",
                "",
                "",
            ])
    return data

data_s_fc_3 = sentence_flashcards(df_7000_sents_3)
df_s_fc_3 = pd.DataFrame(data_s_fc_3, columns=cols)

frames_s_fc = [df_s_fc_3]
df_s_fc = pd.concat(frames_s_fc)
df_s_fc

# Creating Verb Conjugation MCQ Exercises


In [ ]:
def verb_conjug_mcq(df_source, exo_id, cols):
    
    verb_conjug = {col: [] for col in cols}

    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi"][i]))

        count = 0
        blank = []
        answer = []
        hint = []

        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                if word.upos in ["VERB", "AUX"] and word.text != word.lemma:
                    blank.append(count)
                    answer.append(word.text)
                    hint.append(word.lemma)

                count += 1

        if len(blank):
            question = []

            for b in range(len(blank)):

                question.append(
                    " ".join([
                        word.text 
                        if word.id != blank[b] + 1
                        else "..." 
                        for word in sent.words 
                        for sent in doc_hi_phrase.sentences
                    ])
                )

                list_of_distractors = []
                list_of_distractors = spelling_distractors(nlp_hi_stanza(answer[b]), list_of_distractors)

                if list_of_distractors != []:

                    exo_id += 1

                    verb_conjug["Exo_type_id"].append("35")
                    verb_conjug["Exo_type"].append("MCQ")
                    verb_conjug["Exo_objective"].append("Grammar")
                    verb_conjug["Exo_focus"].append(
                        "Verb_Conjugation")
                    verb_conjug["Exo_id"].append(str(exo_id))
                    verb_conjug["Source_format"].append("text")
                    verb_conjug["Target_format"].append("text")
                    verb_conjug["Source_sentence_id"].append(
                        str(df_source["ID"][i]))
                    verb_conjug["Source_word_id"].append("")
                    verb_conjug["Source_lang"].append("French")
                    verb_conjug["Target_lang"].append("Hindi")
                    verb_conjug["Full_sentence"].append(
                        str(df_source["French"][i]))
                    verb_conjug["Instruction"].append(
                        "Conjugate the verb correctly:")
                    verb_conjug["Sentence_w_blank"].append(str(f"{question[b]} ({hint[b]})"))
                    verb_conjug["Right_answer"].append(str(answer[b]))
                    verb_conjug["Dist_1"].append(list_of_distractors[0]),
                    verb_conjug["Dist_2"].append(list_of_distractors[1]),
                    verb_conjug["Dist_3"].append(list_of_distractors[2]),
                    verb_conjug["Explanation"].append("")
                    verb_conjug["Difficulty"].append("")
                    verb_conjug["Remediation"].append("")

    return exo_id, verb_conjug

exo_id = 0

exo_id, data_v_conjug_mcq_1 = verb_conjug_mcq(df_7000_sents_1, exo_id, cols)
df_v_conjug_mcq_1 = pd.DataFrame(data_v_conjug_mcq_1, columns=cols)

exo_id, data_v_conjug_mcq_2 = verb_conjug_mcq(df_7000_sents_2, exo_id, cols)
df_v_conjug_mcq_2 = pd.DataFrame(data_v_conjug_mcq_2, columns=cols)

exo_id, data_v_conjug_mcq_3 = verb_conjug_mcq(df_7000_sents_3, exo_id, cols)
df_v_conjug_mcq_3 = pd.DataFrame(data_v_conjug_mcq_3, columns=cols)

frames_v_conjug_mcq = [df_v_conjug_mcq_1, df_v_conjug_mcq_2, df_v_conjug_mcq_3]
df_v_conjug_mcq = pd.concat(frames_v_conjug_mcq)
df_v_conjug_mcq

# Creating Verb Conjugation Cloze Test Exercises


In [ ]:
def verb_conjug_cloze_test(df_source, exo_id):
    data = []
    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi"][i]))

        count = 0
        blank = []
        answer = []
        hint = []
        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                if word.upos in ["VERB", "AUX"] and word.text != word.lemma:
                    blank.append(count)
                    answer.append(word.text)
                    hint.append(word.lemma)

                count += 1

        if len(blank):
            question = []

            for b in range(len(blank)):
                exo_id += 1
                question.append(
                    " ".join([
                        word.text 
                        if word.id != blank[b] + 1 
                        else "..." 
                        for word in sent.words 
                        for sent in doc_hi_phrase.sentences
                    ])
                )
                data.append([
                    "38", 
                    "Cloze_Test", 
                    "Grammar", 
                    "Verb_Conjugation", 
                    str(exo_id), 
                    "text", 
                    "text", 
                    str(df_source["ID"][i]), 
                    "", 
                    "French", 
                    "Hindi", 
                    str(df_source["Hindi"][i]), 
                    "Conjugate the verb correctly:", 
                    f"{question[b]} ({hint[b]})", 
                    str(answer[b]), 
                    "", 
                    "", 
                    "", 
                    "", 
                    "", 
                    "",
                ])

    return exo_id, data

exo_id = 0

exo_id, data_v_conjug_ct_1 = verb_conjug_cloze_test(df_7000_sents_1, exo_id)
df_v_conjug_ct_1 = pd.DataFrame(data_v_conjug_ct_1, columns=cols)

exo_id, data_v_conjug_ct_2 = verb_conjug_cloze_test(df_7000_sents_2, exo_id)
df_v_conjug_ct_2 = pd.DataFrame(data_v_conjug_ct_2, columns=cols)

exo_id, data_v_conjug_ct_3 = verb_conjug_cloze_test(df_7000_sents_3, exo_id)
df_v_conjug_ct_3 = pd.DataFrame(data_v_conjug_ct_3, columns=cols)

frames_v_conjug_ct = [df_v_conjug_ct_1, df_v_conjug_ct_2, df_v_conjug_ct_3]
df_v_conjug_ct = pd.concat(frames_v_conjug_ct)
df_v_conjug_ct

# Merge All Exercise Dataframes


In [ ]:
frames_hi_exercises = [df_w_fc, df_spelling_mcq, df_s_fc, df_v_conjug_mcq, df_v_conjug_ct]
df_hi_exercises = pd.concat(frames_hi_exercises)
df_hi_exercises

# Extract Exercise Dataframe to an Excel File


In [ ]:
df_hi_exercises.to_excel("Hindi_Exercises.xlsx", index=False)